In [1]:
%pip install mlflow --quiet
%pip install optuna
%pip install mlflow scikit-learn
%pip install imblearn

You should consider upgrading via the '/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 348 kB 384 kB/s eta 0:00:01
     |████████████████████████████████| 78 kB 5.1 MB/s  eta 0:00:01
     |████████████████████████████████| 80 kB 5.1 MB/s eta 0:00:01
     |████████████████████████████████| 41.6 MB 10.0 MB/s eta 0:00:01     |█████████████▋                  | 17.7 MB 12.7 MB/s eta 0:00:02
     |████████████████████████████████| 49 kB 3.4 MB/s eta 0:00:011
     |████████████████████████████████| 147 kB 9.8 MB/s eta 0:00:01
     |████████████████████████████████| 112 kB 77 kB/s eta 0:00:011
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11107 sha256=5b9802bafd6ad2d1c2fda829fce62f18e661516a815c54d8fb91c291da603db9
  Stored in directory: /home/luizkof/.cache/pip/wheels/7f/1a/65

In [1]:
import numpy as np
import pandas as pd
import optuna
import gc
import mlflow

#### Leitura dos dados

In [2]:
data_path = './TRNcod.xls'

In [3]:
df_excel = pd.read_csv(data_path, sep='\t', encoding='utf-8')

In [4]:
print('Quantidade de entradas:', len(df_excel.index))

Quantidade de entradas: 389196


In [5]:
df_excel.head()

,INDEX,UF_1,UF_2,UF_3,UF_4,UF_5,UF_6,UF_7,IDADE,SEXO_1,...,CEP4_7,CEP4_8,CEP4_9,CEP4_10,CEP4_11,CEP4_12,CEP4_13,CEP4_14,IND_BOM_1_1,IND_BOM_1_2
0,0,1,1,1,0,0,0,0,0.135098,1,...,0,0,1,1,0,1,1,1,0,1
1,1,1,0,1,0,0,1,0,0.273504,1,...,0,1,0,1,1,0,0,0,1,0
2,2,1,0,1,0,0,1,0,0.281910,0,...,1,1,0,0,0,0,1,0,1,0
3,3,1,1,1,0,0,0,0,0.225741,0,...,1,1,0,1,1,0,1,0,1,0
4,4,1,1,0,0,0,1,0,0.480403,0,...,1,1,1,0,0,1,0,1,1,0


In [6]:
df_excel.drop(['INDEX'], axis=1, inplace=True)

#### Combinando classes em uma única coluna

In [7]:
df = df_excel

In [8]:
def label_class (row):
   if row['IND_BOM_1_1'] == 1 and row['IND_BOM_1_2'] == 1:
      return 2
   if row['IND_BOM_1_1'] == 1:
      return 0
   if row['IND_BOM_1_2'] == 1:
      return 1
   return 3

In [9]:
df['class'] = df.apply (lambda row: label_class(row), axis=1)

In [10]:
df.head()

,UF_1,UF_2,UF_3,UF_4,UF_5,UF_6,UF_7,IDADE,SEXO_1,NIVEL_RELACIONAMENTO_CREDITO01,...,CEP4_8,CEP4_9,CEP4_10,CEP4_11,CEP4_12,CEP4_13,CEP4_14,IND_BOM_1_1,IND_BOM_1_2,class
0,1,1,1,0,0,0,0,0.135098,1,0.222222,...,0,1,1,0,1,1,1,0,1,1
1,1,0,1,0,0,1,0,0.273504,1,0.111111,...,1,0,1,1,0,0,0,1,0,0
2,1,0,1,0,0,1,0,0.281910,0,1.000000,...,1,0,0,0,0,1,0,1,0,0
3,1,1,1,0,0,0,0,0.225741,0,0.111111,...,1,0,1,1,0,1,0,1,0,0
4,1,1,0,0,0,1,0,0.480403,0,0.111111,...,1,1,0,0,1,0,1,1,0,0


In [11]:
print('classe 0:', len(df[df['class'] == 0]))
print('classe 1:', len(df[df['class'] == 1]))
print('classe desconhecida:', len(df[df['class'] > 1]))

classe 0: 255098
classe 1: 134098
classe desconhecida: 0


#### 1 Etapa - Particionamento dos dados

In [12]:
x_class_0 = df[df['class'] == 0]
x_class_1 = df[df['class'] == 1]

In [13]:
x_class_0.head()

,UF_1,UF_2,UF_3,UF_4,UF_5,UF_6,UF_7,IDADE,SEXO_1,NIVEL_RELACIONAMENTO_CREDITO01,...,CEP4_8,CEP4_9,CEP4_10,CEP4_11,CEP4_12,CEP4_13,CEP4_14,IND_BOM_1_1,IND_BOM_1_2,class
1,1,0,1,0,0,1,0,0.273504,1,0.111111,...,1,0,1,1,0,0,0,1,0,0
2,1,0,1,0,0,1,0,0.281910,0,1.000000,...,1,0,0,0,0,1,0,1,0,0
3,1,1,1,0,0,0,0,0.225741,0,0.111111,...,1,0,1,1,0,1,0,1,0,0
4,1,1,0,0,0,1,0,0.480403,0,0.111111,...,1,1,0,0,1,0,1,1,0,0
5,0,1,1,0,0,0,1,0.219323,0,0.111111,...,1,1,0,1,0,0,1,1,0,0


In [14]:
x_class_1.head()

,UF_1,UF_2,UF_3,UF_4,UF_5,UF_6,UF_7,IDADE,SEXO_1,NIVEL_RELACIONAMENTO_CREDITO01,...,CEP4_8,CEP4_9,CEP4_10,CEP4_11,CEP4_12,CEP4_13,CEP4_14,IND_BOM_1_1,IND_BOM_1_2,class
0,1,1,1,0,0,0,0,0.135098,1,0.222222,...,0,1,1,0,1,1,1,0,1,1
10,1,0,1,1,0,0,0,0.654703,0,0.111111,...,1,0,1,0,0,0,0,0,1,1
11,1,1,1,0,0,0,0,0.097444,1,0.111111,...,1,0,1,0,1,0,1,0,1,1
12,1,0,1,0,1,0,0,0.398961,0,0.111111,...,1,1,0,1,0,0,0,0,1,1
17,1,1,1,0,0,0,0,0.142254,0,0.111111,...,0,0,1,0,1,0,1,0,1,1


#### 2 Etapa - Particionamento dos dados

In [15]:
y_class_0 = x_class_0['class'].values
y_class_1 = x_class_1['class'].values

#### Separar dados em treino, validação e treino

Os dados foram divididos aleatoriamente usando o método train_test_split()
- 50% dos dados para treinamento
- Dos 50% restantes, metade(25% do total) pra validação e o restante(25%) para teste

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
X_train_class_0, X_rem_class_0, y_train_class_0, y_rem_class_0 = train_test_split(x_class_0, y_class_0, train_size=0.5)
X_valid_class_0, X_test_class_0, y_valid_class_0, y_test_class_0 = train_test_split(X_rem_class_0, y_rem_class_0, test_size=0.5)

In [18]:
X_train_class_1, X_rem_class_1, y_train_class_1, y_rem_class_1 = train_test_split(x_class_1, y_class_1, train_size=0.5)
X_valid_class_1, X_test_class_1, y_valid_class_1, y_test_class_1 = train_test_split(X_rem_class_1, y_rem_class_1, test_size=0.5)

In [19]:
X_train = pd.concat([X_train_class_0, X_train_class_1])
y_train = np.concatenate((y_train_class_0, y_train_class_1))
X_train = X_train.drop(['class', 'IND_BOM_1_1', 'IND_BOM_1_2'], axis=1)

In [20]:
X_valid = pd.concat([X_valid_class_0, X_valid_class_1])
y_valid = np.concatenate((y_valid_class_0, y_valid_class_1))
X_valid = X_valid.drop(['class', 'IND_BOM_1_1', 'IND_BOM_1_2'], axis=1)

In [21]:
X_test = pd.concat([X_test_class_0, X_test_class_1])
y_test = np.concatenate((y_test_class_0, y_test_class_1))
X_test = X_test.drop(['class', 'IND_BOM_1_1', 'IND_BOM_1_2'], axis=1)

In [22]:
print('Quantidade de entradas para treino:', len(X_train), len(y_train))
print('Quantidade de entradas para validação:', len(X_valid), len(y_valid))
print('Quantidade de entradas para teste:', len(X_test), len(y_test))

Quantidade de entradas para treino: 194598 194598
Quantidade de entradas para validação: 97298 97298
Quantidade de entradas para teste: 97300 97300


#### Oversampling - Replicar a classe minoritária para ficar do tamanho da classe majoritária

In [23]:
print('Quantidade antes do over sampling')
print('Quantidade classe 0 treino:', (y_train == 0).sum())
print('Quantidade classe 1 treino:', (y_train == 1).sum())
print('Quantidade classe 0 valid:', (y_valid == 0).sum())
print('Quantidade classe 1 valid:', (y_valid == 1).sum())
print('Quantidade classe 0 test:', (y_test == 0).sum())
print('Quantidade classe 1 test:', (y_test == 1).sum())

Quantidade antes do over sampling
Quantidade classe 0 treino: 127549
Quantidade classe 1 treino: 67049
Quantidade classe 0 valid: 63774
Quantidade classe 1 valid: 33524
Quantidade classe 0 test: 63775
Quantidade classe 1 test: 33525


In [24]:
from imblearn.over_sampling import SMOTE

In [25]:
oversample = SMOTE()
X_train, y_train = oversample.fit_resample(X_train, y_train)

In [26]:
print('Quantidade classe 0 treino:', (y_train == 0).sum())
print('Quantidade classe 1 treino:', (y_train == 1).sum())

Quantidade classe 0 treino: 127549
Quantidade classe 1 treino: 127549


In [27]:
oversample = SMOTE()
X_valid, y_valid = oversample.fit_resample(X_valid, y_valid)

In [28]:
print('Quantidade classe 0 valid:', (y_valid == 0).sum())
print('Quantidade classe 1 valid:', (y_valid == 1).sum())

Quantidade classe 0 valid: 63774
Quantidade classe 1 valid: 63774


In [29]:
print('Quantidade classe 0 test:', (y_test == 0).sum())
print('Quantidade classe 1 test:', (y_test == 1).sum())

Quantidade classe 0 test: 63775
Quantidade classe 1 test: 33525


#### Funções auxiliares para métricas

Métricas utilizadas:

- Acurácia

- Recall

- Precision

- F1-Score

- Auroc (Área sob a Curva Roc)

- Matriz de confusão
      [TP  FP]
      [FN  TN]



In [30]:
### VALIDAR: Teste estatístico Kolmogorov-Smirnov -KS (principal)
### TODO: Adiconar alguns plots

In [31]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.metrics import roc_curve, RocCurveDisplay

import matplotlib
import matplotlib.pyplot as plt

In [47]:
def print_metrics(actual, pred, pred_proba):
  print('Accuracy: {:.4f}'.format(accuracy_score(actual, pred)))
  print('Recall: {:.4f}'.format(recall_score(actual, pred)))
  print('Precision: {:.4f}'.format(precision_score(actual, pred)))
  print('F1-Score: {:.4f}'.format(f1_score(actual, pred)))
  if pred_proba:
    print('ROC AUC Score: {:.4f}'.format(roc_auc_score(actual, pred_proba[:, 1])))
  print('Matriz de confusão no conjunto de teste:')
  print(confusion_matrix(actual, pred))

In [33]:
def eval_metrics(actual, pred):
  accuracy = accuracy_score(actual, pred)
  recall = recall_score(actual, pred)
  precision = precision_score(actual, pred)
  f1 = f1_score(actual, pred)
  return accuracy, recall, precision, f1

### Modelo SVM

Descrição dos principais parâmetros:

- C
  - Parâmetro de regularização. A força da regularização é inversamente proporcional a C. Deve ser estritamente positiva. A penalidade é uma penalidade de 12 ao quadrado.

- kernel
  - Especifica o tipo de kernel a ser usado no algoritmo. Se nenhum for fornecido, 'rbf' será usado. Se um callable for fornecido, ele será usado para pré-computar a matriz do kernel a partir de matrizes de dados; essa matriz deve ser uma matriz de forma
  - Valor default: rbf

- degree
  - Grau da função kernel polinomial ('poli'). Ignorado por todos os outros kernels.
  - Valor default: 3

- gamma
  - Coeficiente de kernel para 'rbf', 'poli' e 'sigmóide'


In [38]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
clf_svm = SVC(gamma='auto', probability=True)

In [39]:
%%time
clf_svm.fit(X_train, y_train)

KeyboardInterrupt: 

In [ ]:
y_valid_pred_svm = clf_svm.predict(X_valid)

: 

In [ ]:
y_valid_proba_svm = clf_svm.predict_proba(X_valid)

##### Análise de Desempenho

In [ ]:
print('Métricas da 1ª configuração do SVM:\n')
print_metrics(y_valid, y_valid_pred_svm, y_valid_proba_svm)

**Segundo a própria documentação do Scikit learn para dataset muito grandes é aconselhável usarmos o LinearSVC**

In [39]:
from sklearn.svm import LinearSVC
clf_svm_linear = LinearSVC(random_state=0, tol=1e-5)

In [41]:
%%time
clf_svm_linear.fit(X_train, y_train)

CPU times: user 2min 29s, sys: 599 ms, total: 2min 29s
Wall time: 2min 29s


/home/luizkof/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC(random_state=0, tol=1e-05)

In [42]:
y_valid_pred_svm_linear = clf_svm_linear.predict(X_valid)

##### Análise de Desempenho

In [48]:
print('Métricas da 1ª configuração do SVM SVC Linear:\n')
print_metrics(y_valid, y_valid_pred_svm_linear,None)

Métricas da 1ª configuração do SVM SVC Linear:

Recall: 0.6593
Precision: 0.7668
F1-Score: 0.7090
Matriz de confusão no conjunto de teste:
[[50988 12786]
 [21730 42044]]


### Modelo Random Forest

Descrição dos parâmetros:

- n_estimators
  - O número de árvores na floresta.

- criterion
  - A função para medir a qualidade de uma divisão

- max_depth
  - A profundidade máxima da árvore.
  - 'None' significa que os nós são expandidos até que todas as folhas sejam puras(se o nó possui prediz apenas 1 classe) ou até que todas as folhas contenham menos de min_samples_split amostras.

- min_samples_split
  - O número mínimo de amostras necessárias para dividir um nó.

- min_samples_leaf
  - O número mínimo de amostrar necessárias para ser um nó folha.

- max_features
  - O número de features a serem considerados ao procurar a melhor divisão. Por exemplo, caso a função seja 'sqrt', a cada divisão ele tenta buscar uma condição que possua sqrt(n_node) entradas. 

- max_leaf_nodes
  - O número max de nós folha. 
  - 'None' significa então pode haver um número ilimitado de nós folha.

In [36]:
from sklearn.ensemble import RandomForestClassifier

#### 1ª Configuração

A primeira configuração é utilizada com os parâmetros padrões do Sklearn.

Segue abaixo a lista dos principais hiperparâmetros:

- n_estimators = 100

- criterion = 'gini'

- max_depth = None 

- min_samples_split = 2

- min_samples_leaf = 1

- max_features = 'sqrt'

- max_leaf_nodes = None

In [37]:
clf_rf = RandomForestClassifier(random_state=42, n_jobs=-1)

In [38]:
%%time
history_rf = clf_rf.fit(X_train, y_train)

CPU times: total: 6min 6s
Wall time: 40 s


In [40]:
y_valid_pred_rf = clf_rf.predict(X_valid)

In [41]:
y_valid_proba_rf = clf_rf.predict_proba(X_valid)

##### Análise de Desempenho

In [42]:
print('Métricas da 1ª configuração do Random Forest:\n')
print_metrics(y_valid, y_valid_pred_rf, y_valid_proba_rf)

Métricas da 1ª configuração do Random Forest:

Accuracy: 0.7333
Recall: 0.6483
Precision: 0.7811
F1-Score: 0.7085
ROC AUC Score: 0.8140
Matriz de confusão no conjunto de teste:
[[52191 11583]
 [22432 41342]]


#### 2ª Configuração

Para a segunda configuração vamos criar um estudo de caso usando o optuna, variando alguns hiperparâmetros

In [66]:
mlflow.sklearn.autolog()
mlflow.set_experiment('validacao')

2022/09/23 09:49:51 INFO mlflow.tracking.fluent: Experiment with name 'validacao' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///C:/Users/jonat/Documents/UFPE/redes-neurais/mlruns/1', experiment_id='1', lifecycle_stage='active', name='validacao', tags={}>

In [77]:
def random_forest(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 400),
        'max_depth': trial.suggest_int('max_depth', 4, 100),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 100),
    }
    # Create the model
    with mlflow.start_run(run_name="Random Forest - Validacao"):
        rnd_forest = RandomForestClassifier(
            n_estimators=params["n_estimators"],
            max_depth=params["max_depth"],
            min_samples_split=params["min_samples_split"],
            random_state=42,
            n_jobs=-1
        )

        rnd_forest.fit(X_train, y_train)

        y_pred_valid = rnd_forest.predict(X_valid)
        y_pred_proba = rnd_forest.predict_proba(X_valid)

        (accuracy, recall, precision, f1) = eval_metrics(y_valid, y_pred_valid)

        mlflow.log_params(params)
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("f1", f1)

        gc.collect()
        return accuracy

In [78]:
rf = optuna.create_study(direction="maximize")
rf.optimize(random_forest, n_trials=10)

[I 2022-09-23 11:42:52,375] A new study created in memory with name: no-name-a6b8ffed-5489-496c-a98e-fecfefd91933
2022/09/23 11:43:40 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\jonat\anaconda3\lib\site-packages\mlflow\models\signature.py:129: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
[I 2022-09-23 11:43:48,221] Trial 0 fin

### MLP

Descrição dos parâmetros:

- n_estimators
  - O número de árvores na floresta.


In [82]:
%pip install tensorflow

^C
Note: you may need to restart the kernel to use updated packages.


In [83]:
from keras.wrappers.scikit_learn import KerasClassifier

from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping

#### 1ª Configuração

A primeira configuração é utilizada com os parâmetros padrões do Sklearn.

Segue abaixo a lista dos principais hiperparâmetros:

- n_

In [60]:
def create_sklearn_compatible_model():
    input_dim = X_train.shape[1]
    
    model = Sequential()
    model.add(Dense(20, activation='tanh', input_dim=input_dim))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [61]:
clf_mlp = KerasClassifier(build_fn=create_sklearn_compatible_model, 
                          batch_size=64, epochs=100,
                          verbose=0)


/tmp/ipykernel_4606/3075366852.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  clf_mlp = KerasClassifier(build_fn=create_sklearn_compatible_model,


In [62]:
%%time
clf_mlp.fit(X_train, y_train)

2022-09-22 23:54:28.323084: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 495910512 exceeds 10% of free system memory.


CPU times: user 8min 1s, sys: 43.6 s, total: 8min 45s
Wall time: 5min 41s


In [63]:
y_pred_mlp = clf_mlp.predict(X_valid)

  91/3986 [..............................] - ETA: 4s

2022-09-23 00:00:08.716645: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 247953312 exceeds 10% of free system memory.


3986/3986 [==============================] - 4s 908us/step


In [64]:
y_proba_mlp = clf_mlp.predict_proba(X_valid)

  35/3986 [..............................] - ETA: 17s

2022-09-23 00:00:14.785572: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 247953312 exceeds 10% of free system memory.


3986/3986 [==============================] - 4s 921us/step


##### Análise de Desempenho

In [65]:
print('Métricas da 1ª configuração do Random Forest:\n')
print_metrics(y_valid, y_pred_mlp, y_proba_mlp)

Métricas da 1ª configuração do Random Forest:

Accuracy: 0.7325
Recall: 0.6387
Precision: 0.7862
F1-Score: 0.7048
ROC AUC Score: 0.8154
Matriz de confusão no conjunto de teste:
[[52694 11080]
 [23039 40735]]


#### 2ª Configuração

Para a segunda configuração alteramos os seguintes parâmetros:

### Gradient Boosting

Descrição dos parâmetros:

- loss
  - Possíveis valores: ‘log_loss’, ‘exponential’
  - default=’log_loss’
  - A função de perda a ser otimizada. 
    - 'log_loss' refere-se ao desvio binomial e multinomial, o mesmo usado na regressão logística. É uma boa escolha para classificação com saídas probabilísticas. 
    - 'exponencial', o aumento de gradiente recupera o algoritmo AdaBoost.

- learning_rate
  - Possíveis valores: intervalo (0,0, inf)
  - default=0.1
  - A taxa de aprendizado reduz a contribuição de cada árvore por learning_rate. Há um trade-off entre learning_rate e n_estimators. 

- n_estimators
  - Os valores devem estar no intervalo [1, inf).
  - default=100
  - O número de estágios de reforço a serem executados. 
  - **O aumento de gradiente é bastante robusto ao over-fitting, portanto, um número grande geralmente resulta em melhor desempenho.

- subsample
  - Os valores devem estar no intervalo (0,0, 1,0].
  - default=1.0
  - A fração de amostras a ser usada para ajustar os 'individual base learners'. 
  - Se menor que 1,0, isso resulta em aumento de gradiente estocástico. subamostra interage com o parâmetro n_estimators. 
    - **Escolher subamostra < 1,0 leva a uma redução da variância e a um aumento no viés.

- criterion
  - Possíveis valores: {‘friedman_mse’, ‘squared_error’, ‘mse’}
  - default=’friedman_mse’
  - A função para medir a qualidade de uma divisão. 
  - Os critérios suportados são:
    - ‘friedman_mse’ para o erro quadrático médio com pontuação de melhoria por Friedman, 
    - ‘squared_error’ para erro quadrático médio. 
  - **O valor padrão de ‘friedman_mse’ geralmente é o melhor, pois pode fornecer uma melhor aproximação em alguns casos.

- min_samples_split
  - Possíveis valores: int or float
  - default=2
  - Se int, os valores devem estar no intervalo [1, inf).
  - Se float, os valores devem estar no intervalo (0.0, 1.0] e min_samples_leaf será ceil(min_samples_leaf * n_samples).

- min_samples_leaf
  - Possíveis valores: int or float
  - default=1
  - Isso pode ter o efeito de suavizar o modelo, especialmente na regressão.

- min_weight_fraction_leaf
  - Os valores devem estar no intervalo [0,0, 0,5].
  - default=0.0(As amostras têm peso igual)
  - A fração ponderada mínima da soma total de pesos (de todas as amostras de entrada) necessária para estar em um nó folha. 

- max_depth
  - Os valores devem estar no intervalo [1, inf).
  - default=3
  - Ajuste este parâmetro para melhorar o desempenho; 
    - O melhor valor depende da interação das variáveis de entrada.

- min_impurity_decrease
  - Os valores devem estar no intervalo [0,0, inf).
  - default=0.0
  - Um nó será dividido se esta divisão induzir uma diminuição da impureza maior ou igual a este valor.

- init
  - Possíveis valores: estimator or ‘zero’
  - default=None(é usado um DummyEstimator)
  - Um objeto estimador que é usado para calcular as previsões iniciais. 
  - init tem que fornecer fit e predict_proba. 
  - Se 'zero', as previsões brutas iniciais são definidas como zero. 

- max_features
  - Possíveis valores: {‘auto’, ‘sqrt’, ‘log2’}, int or float
    - Se int, valores devem estar no intervalo  [1, inf).
    - Se float, valores devem estar no intervalo  (0.0, 1.0] and the features considered at each split will be max(1, int(max_features * n_features_in_)).
    - Se f = ‘auto’, ‘sqrt’, ‘log2’, então max_features = f(n_features).
    - Se None, então max_features = n_features.
  - default=None
  - O número de features para considerar quando buscar pelo melhor split.



In [84]:
from sklearn.ensemble import GradientBoostingClassifier

#### 1ª Configuração

A primeira configuração é utilizada com os parâmetros padrões do Sklearn.

Segue abaixo a lista dos principais hiperparâmetros:

- loss
  - default=’log_loss’

- learning_rate
  - default=0.1

- n_estimators
  - default=100

- subsample
  - default=1.0

- criterion
  - default=’friedman_mse’

- min_samples_split
  - default=2

- min_samples_leaf
  - default=1

- min_weight_fraction_leaf
  - default=0.0(As amostras têm peso igual)

- max_depth
  - default=3

- min_impurity_decrease
  - default=0.0

- init
  - default=None(é usado um DummyEstimator)

- max_features
  - default=None(então max_features = n_features.)




In [89]:
clf_gb = GradientBoostingClassifier(random_state=27)

In [90]:
clf_gb.fit(X_train, y_train)

2022/09/23 13:26:38 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'ca2f511133b8448cbd529bd2a18fb4e5', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow


KeyboardInterrupt: 

In [76]:
y_pred_gb = clf_gb.predict(X_valid)

In [77]:
y_proba_gb = clf_gb.predict_proba(X_valid)

##### Análise de Desempenho

In [78]:
print('Métricas da 1ª configuração do Random Forest:\n')
print_metrics(y_valid, y_pred_gb, y_proba_gb)

Métricas da 1ª configuração do Random Forest:

Accuracy: 0.7494
Recall: 0.6769
Precision: 0.7917
F1-Score: 0.7298
ROC AUC Score: 0.8323
Matriz de confusão no conjunto de teste:
[[52416 11358]
 [20607 43167]]


#### 2ª Configuração

Para a segunda configuração alteramos os seguintes parâmetros:

In [ ]:
def gradient_boosting(trial):
    params = {
        'learning_rate': trial.suggest_float('learning_rate ', 0.0001, 0.1, step=0.005),
        'max_depth': trial.suggest_int('max_depth', 4, 100),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 100),
    }
    # Create the model
    with mlflow.start_run(run_name="Gradient Boosting - Validacao"):
        gb = GradientBoostingClassifier(
            n_estimators=params["n_estimators"],
            max_depth=params["max_depth"],
            min_samples_split=params["min_samples_split"],
            random_state=42,
            n_jobs=-1
        )

        gb.fit(X_train, y_train)

        y_pred_valid = gb.predict(X_valid)
        y_pred_proba = gb.predict_proba(X_valid)

        (accuracy, recall, precision, f1) = eval_metrics(y_valid, y_pred_valid)

        mlflow.log_params(params)
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("f1", f1)

        gc.collect()
        return accuracy